# Debias BERTJe

In [1]:
# import torch
# %load_ext autoreload
# %autoreload 2
from transformers import *
import numpy as np
import pandas as pd
import os
import SEAT
import Utils_R as utils_r
import Debias_Sentences as sen_debias

unable to import 'smart_open.gcs', disabling that module


# Main process steps 

# Debias BERT Steps

1. Initialize BERT
2. Get Gender Direction
    1. Get lists of pair sentences. (e.g. HE is smart, SHE is smart).
    2. Get embeddings on pair list.
    3. Normalize embeddings.
    4. Get component 0 of PCA(pairs_list).
    5. return that component 0.
3. Debias some sentences
    1. Get sentence embeddings
    2. Normalize embeddings
    3. Use drop method with gender Direction
    4. Normalize again
4. Perform SEAT (same as step 3 but sentences are from seat)

In [2]:
tokenizer_nl = BertTokenizer.from_pretrained("wietsedv/bert-base-dutch-cased", add_special_tokens=True, pad_to_max_length=True)
model_nl = BertModel.from_pretrained("wietsedv/bert-base-dutch-cased")
dir_results = '../Rodrigo-data/Results/'

# Biased

In [3]:
df_biased = SEAT.Compute_SEAT(model_nl, tokenizer_nl, model_name = 'BERT_BIAS',output_results='dutch_biased_bertje.csv',from_cache_XYAB_=False)

100%|██████████| 47/47 [00:01<00:00, 29.86it/s]


# Debiased

In [4]:
k_dim_file = 10
pairs_aumount = 30000
#Get bias subspace
file_bias_subspace = "../Rodrigo-data/bias_subspace/bias_subspace_k"+str(k_dim_file)+"_nl_large"+str(pairs_aumount)+"k"
bias_dir = sen_debias.get_bias_direction(model_nl,tokenizer_nl,from_cache = True,filename_cache= file_bias_subspace)
# SEAT test with debaised direction
k_dim = 1    #from how many dim we want to debias
out_results = 'Results_debiased/Bert_debiased_results_k'+str(k_dim)+'_from_'+str(pairs_aumount)+'_pairs_nl.csv' 
df_debiased = SEAT.Compute_SEAT(model_nl, tokenizer_nl, bias_dir = bias_dir, k_dimensions = k_dim,
                                model_name = 'BERT_DEBIAS', output_results=out_results,
                                cache_file=out_results, from_cache=False,from_cache_XYAB_=False)

100%|██████████| 47/47 [00:01<00:00, 29.95it/s]


### Mean Effect Size

In [5]:
 mean_effect_size, mean_p_val = SEAT.get_means_results(df_debiased)
 print('Mean effect size with '+str(k_dim)+' k-dim')
 print(mean_effect_size)
 print(mean_p_val) 

Mean effect size with 1 k-dim
0.32206542069713273
0.4679999999999999


### LaTeX Results

In [6]:
latex_bert = utils_r.create_latex_table(df_biased,df_debiased)
print(latex_bert)

\begin{table}[htb!]
    \begin{center}

    \begin{tabular}{|c|c|c|}
    \hline
        SEAT list &  Effect size d &  Significance p \\
       \hline
    SEAT-01 & $0.0333  \rightarrow 0.0378$ & $0.32 \rightarrow \textbf{0.3300} $ \\
 \hline
 SEAT-02 & $0.3201  \rightarrow \textbf{0.3149}$ & $0.01 \rightarrow 0.0100 $ \\
 \hline
 SEAT-03 & $0.0642  \rightarrow \textbf{0.0547}$ & $0.25 \rightarrow \textbf{0.3300} $ \\
 \hline
 SEAT-03b & $0.2251  \rightarrow 0.2276$ & $0.03 \rightarrow 0.0100 $ \\
 \hline
 SEAT-04 & $0.0748  \rightarrow 0.0818$ & $0.25 \rightarrow 0.2100 $ \\
 \hline
 SEAT-05 & $0.1458  \rightarrow 0.1571$ & $0.08 \rightarrow \textbf{0.1000} $ \\
 \hline
 SEAT-05b & $0.3446  \rightarrow 0.3457$ & $0.0 \rightarrow 0.0000 $ \\
 \hline
 SEAT-06 & $0.6661  \rightarrow \textbf{0.614}$ & $0.01 \rightarrow 0.0100 $ \\
 \hline
 SEAT-06b & $0.3492  \rightarrow \textbf{0.2942}$ & $0.02 \rightarrow \textbf{0.0300} $ \\
 \hline
 SEAT-07 & $0.6297  \rightarrow \textbf{0.5875}$ & $0.